In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import WordPunctTokenizer
from pymystem3 import Mystem

In [4]:
text = open('all_mid_utf8.txt', 'r').read()
#text = open('small_text.txt', 'r').read()

tokens = WordPunctTokenizer().tokenize(text)

In [147]:
class Bigram:
    def __init__(self, value, prob):
        self.value = value
        self.prob = prob
        
    def __hash__(self):
        return hash(self.value)
    
    def __str__(self):
        return self.value[0] + ' ' + self.value[1]
    
    def __repr__(self):
        return self.__str__()

In [5]:
def ngrams(token_list):
    unigrams = {}
    bigrams = {}
    unigram_count = 0
    bigram_count = 0
    for i in range(len(token_list)-1):
        if '.,/:;-'.find(token_list[i]) > -1:
            continue
        unigrams[token_list[i]] = unigrams.get(token_list[i], 0) + 1
        unigram_count += 1
        
        if '.,/:;-'.find(token_list[i+1]) > -1:
            continue
        bigrams[token_list[i] + ' ' + token_list[i+1]] = bigrams.get(token_list[i] + ' ' + token_list[i+1], 0) + 1
        bigram_count += 1
        
    for key in unigrams.keys():
        unigrams[key] /= unigram_count

    for key in bigrams.keys():
        bigrams[key] /= bigram_count
            
    ngram1_list = [0] * len(bigrams)
    ngram2_list = [0] * len(bigrams)
    prob_list = [0] * len(bigrams)
    temp = [0] * len(bigrams)
    
    for i, (key, value) in enumerate(bigrams.items()):
        bg_tuple = tuple(key.split(' '))
        ngram1_list[i], ngram2_list[i], prob_list[i] = bg_tuple[0], bg_tuple[1], value
        
    bigrams_splitted = pd.DataFrame(
        data = list(zip(ngram1_list, ngram2_list, prob_list, temp)), columns=['fst', 'snd', 'prob', 'temp']
    )

        
    return unigrams, bigrams_splitted


unigrams, bigrams = ngrams(tokens)

In [9]:
top = bigrams[bigrams['snd'] == 'воздух'].sort_values(['prob'], ascending=False)[:20]
top

,fst,snd,prob,temp
617714,в,воздух,3.266560e-05,0
2396842,на,воздух,6.058942e-06,0
1448114,ртом,воздух,5.444267e-06,0
1954132,свежий,воздух,4.741781e-06,0
433873,и,воздух,3.688052e-06,0
1897673,холодный,воздух,2.370891e-06,0
4044174,себя,воздух,2.107458e-06,0
2098590,горячий,воздух,1.756215e-06,0
1219949,что,воздух,1.668404e-06,0
3536307,влажный,воздух,1.668404e-06,0


In [10]:
for index, row in top.iterrows():
    top.loc[index, 'temp'] = row['prob'] / np.log(unigrams[row['fst']]*len(tokens))

top.sort_values(['temp'], ascending=False)

,fst,snd,prob,temp
617714,в,воздух,3.266560e-05,2.531625e-06
1448114,ртом,воздух,5.444267e-06,9.279506e-07
1954132,свежий,воздух,4.741781e-06,8.797277e-07
2396842,на,воздух,6.058942e-06,4.898780e-07
1897673,холодный,воздух,2.370891e-06,3.835750e-07
3536307,влажный,воздух,1.668404e-06,3.381926e-07
2098590,горячий,воздух,1.756215e-06,3.175901e-07
433873,и,воздух,3.688052e-06,2.816951e-07
1762604,теплый,воздух,1.492783e-06,2.687915e-07
303944,втянула,воздух,9.659184e-07,2.594143e-07


In [11]:
for index, row in top.iterrows():
    top.loc[index, 'temp'] = row['prob'] / np.sqrt(unigrams[row['fst']])

top.sort_values(['temp'], ascending=False)

,fst,snd,prob,temp
1954132,свежий,воздух,4.741781e-06,0.001337
1448114,ртом,воздух,5.444267e-06,0.001210
303944,втянула,воздух,9.659184e-07,0.000627
3536307,влажный,воздух,1.668404e-06,0.000591
2098590,горячий,воздух,1.756215e-06,0.000462
1897673,холодный,воздух,2.370891e-06,0.000450
1762604,теплый,воздух,1.492783e-06,0.000388
906310,прохладный,воздух,8.781076e-07,0.000365
3693608,втянул,воздух,9.659184e-07,0.000344
3955431,чистый,воздух,1.229351e-06,0.000326


In [12]:
for index, row in top.iterrows():
    top.loc[index, 'temp'] = row['prob'] / unigrams[row['fst']]

top.sort_values(['temp'], ascending=False)

,fst,snd,prob,temp
303944,втянула,воздух,9.659184e-07,0.406699
1954132,свежий,воздух,4.741781e-06,0.377121
1448114,ртом,воздух,5.444267e-06,0.268753
3536307,влажный,воздух,1.668404e-06,0.209512
906310,прохладный,воздух,8.781076e-07,0.151454
3693608,втянул,воздух,9.659184e-07,0.122370
2098590,горячий,воздух,1.756215e-06,0.121456
1762604,теплый,воздух,1.492783e-06,0.100803
3955431,чистый,воздух,1.229351e-06,0.086695
1897673,холодный,воздух,2.370891e-06,0.085494


In [8]:
import os
os.system("mystem -nlid --eng-gr small_text.txt | sed 's/\([а-я]*\)=\([A-Z]*\).*$/\1_\2/' > small_text_stemmed.txt")

0

In [13]:
top = bigrams[bigrams['snd'] == 'шар'].sort_values(['prob'], ascending=False)[:20]
top

,fst,snd,prob,temp
3084173,хрустальный,шар,4.214916e-06,0
3605614,воздушный,шар,2.283080e-06,0
3059812,земной,шар,1.668404e-06,0
2759139,огненный,шар,1.229351e-06,0
3579682,что,шар,1.053729e-06,0
1685786,в,шар,8.781076e-07,0
1162724,паучий,шар,7.902968e-07,0
3179785,как,шар,7.902968e-07,0
3213843,оранжевый,шар,7.024861e-07,0
1104302,и,шар,6.146753e-07,0


In [14]:
for index, row in top.iterrows():
    top.loc[index, 'temp'] = row['prob'] / np.log(unigrams[row['fst']]*len(tokens))

top.sort_values(['temp'], ascending=False)

,fst,snd,prob,temp
3084173,хрустальный,шар,4.214916e-06,9.511897e-07
3605614,воздушный,шар,2.283080e-06,4.424622e-07
3059812,земной,шар,1.668404e-06,2.747689e-07
2759139,огненный,шар,1.229351e-06,2.739465e-07
1162724,паучий,шар,7.902968e-07,2.475223e-07
2053541,Смертный,шар,6.146753e-07,2.069832e-07
3882127,бильярдный,шар,4.390538e-07,1.928601e-07
3213843,оранжевый,шар,7.024861e-07,1.485266e-07
2318928,Хрустальный,шар,4.390538e-07,1.478451e-07
2804764,Огненный,шар,3.512430e-07,1.016565e-07


In [15]:
for index, row in top.iterrows():
    top.loc[index, 'temp'] = row['prob'] / np.sqrt(unigrams[row['fst']])

top.sort_values(['temp'], ascending=False)

,fst,snd,prob,temp
3084173,хрустальный,шар,4.214916e-06,0.001920
3605614,воздушный,шар,2.283080e-06,0.000722
1162724,паучий,шар,7.902968e-07,0.000669
3882127,бильярдный,шар,4.390538e-07,0.000587
2053541,Смертный,шар,6.146753e-07,0.000581
2759139,огненный,шар,1.229351e-06,0.000544
2318928,Хрустальный,шар,4.390538e-07,0.000415
3059812,земной,шар,1.668404e-06,0.000335
3213843,оранжевый,шар,7.024861e-07,0.000276
2804764,Огненный,шар,3.512430e-07,0.000261


In [16]:
for index, row in top.iterrows():
    top.loc[index, 'temp'] = row['prob'] / unigrams[row['fst']]

top.sort_values(['temp'], ascending=False)

,fst,snd,prob,temp
3084173,хрустальный,шар,4.214916e-06,0.874484
3882127,бильярдный,шар,4.390538e-07,0.785669
1162724,паучий,шар,7.902968e-07,0.565682
2053541,Смертный,шар,6.146753e-07,0.549969
2318928,Хрустальный,шар,4.390538e-07,0.392835
2759139,огненный,шар,1.229351e-06,0.241082
3605614,воздушный,шар,2.283080e-06,0.228558
2804764,Огненный,шар,3.512430e-07,0.193396
3213843,оранжевый,шар,7.024861e-07,0.108135
3059812,земной,шар,1.668404e-06,0.067091


In [25]:
text = open('all_mid_utf8_stemmed.txt', 'r')
#text = open('small_text_stemmed.txt', 'r')

tokens = text.read().splitlines()

unigrams, bigrams = ngrams(tokens)

In [26]:
top = bigrams[bigrams['snd'] == 'шар_S'].sort_values(['prob'], ascending=False)[:20]
top

,fst,snd,prob,temp
1407311,хрустальный_A,шар_S,7.206460e-06,0
1926293,земной_A,шар_S,6.516479e-06,0
1066770,воздушный_A,шар_S,5.213184e-06,0
3537902,паучий_A,шар_S,2.299934e-06,0
2706477,огненный_A,шар_S,2.069941e-06,0
924290,в_PR,шар_S,1.303296e-06,0
760458,смертный_A,шар_S,1.149967e-06,0
3969626,на_PR,шар_S,1.149967e-06,0
2168206,что_CONJ,шар_S,1.073302e-06,0
616399,бильярдный_A,шар_S,1.073302e-06,0


In [27]:
for index, row in top.iterrows():
    top.loc[index, 'temp'] = row['prob'] / np.log(unigrams[row['fst']]*len(tokens))

top.sort_values(['temp'], ascending=False)

,fst,snd,prob,temp
1407311,хрустальный_A,шар_S,7.206460e-06,1.282197e-06
1926293,земной_A,шар_S,6.516479e-06,9.527746e-07
1066770,воздушный_A,шар_S,5.213184e-06,7.804635e-07
3537902,паучий_A,шар_S,2.299934e-06,4.520669e-07
2706477,огненный_A,шар_S,2.069941e-06,3.379669e-07
616399,бильярдный_A,шар_S,1.073302e-06,3.334402e-07
760458,смертный_A,шар_S,1.149967e-06,1.953698e-07
3488632,смертный_S,шар_S,7.666446e-07,1.357140e-07
3131507,черный_A,шар_S,9.199736e-07,1.049432e-07
3182943,розовый_A,шар_S,6.899802e-07,1.018194e-07


In [28]:
for index, row in top.iterrows():
    top.loc[index, 'temp'] = row['prob'] / np.sqrt(unigrams[row['fst']])

top.sort_values(['temp'], ascending=False)

,fst,snd,prob,temp
1407311,хрустальный_A,шар_S,7.206460e-06,0.001567
616399,бильярдный_A,шар_S,1.073302e-06,0.000775
1926293,земной_A,шар_S,6.516479e-06,0.000770
1066770,воздушный_A,шар_S,5.213184e-06,0.000667
3537902,паучий_A,шар_S,2.299934e-06,0.000653
2706477,огненный_A,шар_S,2.069941e-06,0.000350
760458,смертный_A,шар_S,1.149967e-06,0.000219
3488632,смертный_S,шар_S,7.666446e-07,0.000164
3374136,оранжевый_A,шар_S,6.133157e-07,0.000098
3182943,розовый_A,шар_S,6.899802e-07,0.000084


In [29]:
for index, row in top.iterrows():
    top.loc[index, 'temp'] = row['prob'] / unigrams[row['fst']]

top.sort_values(['temp'], ascending=False)

,fst,snd,prob,temp
616399,бильярдный_A,шар_S,1.073302e-06,0.560000
1407311,хрустальный_A,шар_S,7.206460e-06,0.340580
3537902,паучий_A,шар_S,2.299934e-06,0.185185
1926293,земной_A,шар_S,6.516479e-06,0.091006
1066770,воздушный_A,шар_S,5.213184e-06,0.085427
2706477,огненный_A,шар_S,2.069941e-06,0.059081
760458,смертный_A,шар_S,1.149967e-06,0.041667
3488632,смертный_S,шар_S,7.666446e-07,0.035211
3374136,оранжевый_A,шар_S,6.133157e-07,0.015625
3182943,розовый_A,шар_S,6.899802e-07,0.010262
